In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# top 40 dax companies 
dax_assets = {
               'DAX': '^GDAXI',
               'Linde': 'LIN',
               'SAP': 'SAP',
               'Deutsche Telekom': 'DTE.DE',
               'Volkswagen': 'VOW3.DE',
               'Siemens': 'SIE.DE',
               'Merck': 'MRK.DE',
               'Airbus': 'AIR.PA',
               'Mercedes Benz': 'MBG.DE', 
               'Bayer': 'BAYZF',
               'BMW': 'BMW.DE',
               'Siemens Healthineers': 'SHL.DE',
               'Deutsche Post': 'DPW.DE',
               'BASF': 'BAS.DE',
               'Münchner Rück': 'MUV2.DE',
               'Infineon': 'IFX.DE',
               'Deutsche Börse': 'DB1:DE',
               'RWE': 'RWE.DE',
               'Henkel': 'HEN3.DE',
               'Adidas': 'ADS.DE',
               'Sartorius': 'SRT.DE',
               'Beiersdorf': 'BEI.DE',
               'Porsche': 'PAH3.DE',
               'E.ON': 'EOAN.DE',
               'Deutsche Bank': 'DB',
               'Vonovia': 'VNA.DE',
               'Fresenius': 'FRE.DE',
               'Symrise': 'SY1.DE',
               'Continental': 'CON.DE',
               'Delivery Hero': 'DHER.F',
               'Brenntag': 'BNR.DE',
               'Qiagen': 'QGEN',
               'Fresenius Medical Care': 'FMS',
               'Siemens Energy': 'ENR.F',
               'HeidelbergCement': 'HEI.DE',
               'Puma': 'PUM.DE',
               'MTU Aero Engines': 'MTX.DE',
               'Covestro': '1COV.F',
               'Zalando': 'ZAL.DE',
               'HelloFresh': 'HFG.DE'
            }

In [ ]:
mixed_assets = {
               'DAX': '^GDAXI',
               'Eurostoxx': '^STOXX50E',
               'DowJones': '^DJI',
               'Nikkei': '^N225',
               'SP500': '^GSPC',
               'GOLD': 'GC=F',
               'Silver': 'SI=F',
               'Crude Oil': 'CL=F',
               'Bitcoin': 'BTC-EUR',
            }

In [3]:
dax_top_40_companies = list(dax_assets.keys())#[1:]
dax_top_40_tickername = list(dax_assets.values())#[1:]

In [4]:
start_date = datetime(2019,1,1)
str(start_date.year)+'-'+str(start_date.month)+'-'+str(start_date.day)

'2019-1-1'

In [5]:
start_date = '2019-1-1'
end_date = '2022-9-15'
na_percentage = 0.6 # at least x percent rows must be none-nas

# download DAX top40
df = yf.download(dax_top_40_tickername, 
                    start=start_date, 
                    end=end_date,
                    progress=True)
# drop columns
df = df['Close']
# rename columns
df.columns = dax_top_40_companies
# make sure the index is datetime format
df.index = pd.to_datetime(df.index)
# drop nas
datetimeFormat = '%Y-%m-%d'
time_delta = datetime.strptime(end_date, datetimeFormat) - datetime.strptime(start_date,datetimeFormat)
df.dropna(axis=1, thresh=int(time_delta.days * na_percentage), inplace=True)
# show data
df.head()

[*********************100%***********************]  40 of 40 completed

1 Failed download:
- DB1:DE: No data found, symbol may be delisted


,DAX,Linde,SAP,Deutsche Telekom,Volkswagen,Siemens,Merck,Airbus,Mercedes Benz,Bayer,...,Brenntag,Qiagen,Fresenius Medical Care,Siemens Energy,HeidelbergCement,Puma,MTU Aero Engines,Covestro,Zalando,HelloFresh
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,69.500000,NaN,NaN,NaN,NaN,8.15,...,NaN,99.550003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,42.900002,184.399994,84.169998,60.720001,69.599998,91.860001,69.739998,38.259998,120.599998,8.20,...,19.299999,99.180000,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-03,42.939999,183.949997,81.209999,60.020000,71.510002,91.360001,69.050003,36.630001,121.400002,8.09,...,19.525000,95.459999,35.700001,95.449997,92.599998,64.459999,40.000000,134.759995,23.969999,10416.660156
2019-01-04,45.549999,191.050003,85.120003,63.189999,73.750000,92.180000,71.709999,38.599998,126.000000,8.52,...,19.924999,98.739998,36.669998,99.190002,95.400002,66.540001,40.400002,140.479996,24.760000,10767.690430
2019-01-07,45.150002,189.699997,83.559998,62.900002,74.699997,91.279999,72.120003,38.450001,126.400002,8.70,...,19.900000,99.709999,36.224998,97.889999,99.199997,66.620003,40.580002,140.639999,26.200001,10747.809570


In [6]:
# check if any NAs 
df.isna().sum()

DAX                       19
Linde                     19
SAP                        8
Deutsche Telekom          19
Volkswagen                26
Siemens                   19
Merck                     19
Airbus                    19
Mercedes Benz             19
Bayer                     26
Siemens Healthineers      19
Deutsche Post             19
BASF                      19
Infineon                  19
Deutsche Börse            26
RWE                       19
Henkel                    19
Adidas                    19
Sartorius                 19
Beiersdorf                19
Porsche                   26
E.ON                      19
Deutsche Bank             19
Vonovia                   19
Fresenius                 19
Symrise                   19
Continental               19
Delivery Hero             26
Brenntag                  19
Qiagen                    26
Fresenius Medical Care    19
Siemens Energy            19
HeidelbergCement          19
Puma                      19
MTU Aero Engin

In [7]:
df = df.interpolate(method='time', limit=7).fillna(value=None, method='bfill', axis=0, inplace=False, limit=7, downcast=None)

In [8]:
# check if any NAs left
df.isna().sum()

DAX                       0
Linde                     0
SAP                       0
Deutsche Telekom          0
Volkswagen                0
Siemens                   0
Merck                     0
Airbus                    0
Mercedes Benz             0
Bayer                     0
Siemens Healthineers      0
Deutsche Post             0
BASF                      0
Infineon                  0
Deutsche Börse            0
RWE                       0
Henkel                    0
Adidas                    0
Sartorius                 0
Beiersdorf                0
Porsche                   0
E.ON                      0
Deutsche Bank             0
Vonovia                   0
Fresenius                 0
Symrise                   0
Continental               0
Delivery Hero             0
Brenntag                  0
Qiagen                    0
Fresenius Medical Care    0
Siemens Energy            0
HeidelbergCement          0
Puma                      0
MTU Aero Engines          0
Covestro            

In [9]:
df.head()

,DAX,Linde,SAP,Deutsche Telekom,Volkswagen,Siemens,Merck,Airbus,Mercedes Benz,Bayer,...,Brenntag,Qiagen,Fresenius Medical Care,Siemens Energy,HeidelbergCement,Puma,MTU Aero Engines,Covestro,Zalando,HelloFresh
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,42.900002,184.399994,84.169998,60.720001,69.500000,91.860001,69.739998,38.259998,120.599998,8.15,...,19.299999,99.550003,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-02,42.900002,184.399994,84.169998,60.720001,69.599998,91.860001,69.739998,38.259998,120.599998,8.20,...,19.299999,99.180000,36.810001,98.080002,96.000000,66.120003,39.889999,136.259995,22.959999,10580.190430
2019-01-03,42.939999,183.949997,81.209999,60.020000,71.510002,91.360001,69.050003,36.630001,121.400002,8.09,...,19.525000,95.459999,35.700001,95.449997,92.599998,64.459999,40.000000,134.759995,23.969999,10416.660156
2019-01-04,45.549999,191.050003,85.120003,63.189999,73.750000,92.180000,71.709999,38.599998,126.000000,8.52,...,19.924999,98.739998,36.669998,99.190002,95.400002,66.540001,40.400002,140.479996,24.760000,10767.690430
2019-01-07,45.150002,189.699997,83.559998,62.900002,74.699997,91.279999,72.120003,38.450001,126.400002,8.70,...,19.900000,99.709999,36.224998,97.889999,99.199997,66.620003,40.580002,140.639999,26.200001,10747.809570


In [10]:
# plot daily portfolio worth
fig = px.line(df, 
              y="DAX", 
              title='DAX')
fig.show()

In [11]:
# normalized daily returns
def normalized_returns(df):
    return np.log(1 + df.pct_change(periods=1).fillna(value=None, method='bfill', axis=0, inplace=False, limit=7, downcast=None)) 

# create weights
def weight_creator(df):
    rand = np.random.random(len(df.columns))
    rand /= rand.sum()
    return rand

# calculate portfolio return
def portfolio_returns(df, weights):
    return np.dot(df.mean(), weights)

# calculate portfolios standard deviation
def portfolio_std(df, weights):
    return (np.dot(np.dot(df.cov(), weights), weights))**(1/2)*np.sqrt(250)

def portfolio_sharp_ratio(portfolio_return:float, portfolio_std:float, rfr:float)->float:
    """Calculate the sharp ratio for a given portfolio df and a given risk-free-return "rfr"."""
    ntd = 250 #  number of trading days
    return np.divide(portfolio_return - rfr/ntd, portfolio_std) 

In [12]:
# plot daily portfolio worth
fig = px.line(normalized_returns(df), 
              y="DAX", 
              title='DAX: Normalized Returns')
fig.show()

In [13]:
# testing the functions
df_returns = normalized_returns(df)
weights = weight_creator(df_returns)
sdev = portfolio_std(df_returns, weights)
returns = portfolio_returns(df_returns, weights)

print('portfolio return: {:.5f}'.format(returns))
print('portfolio standard deviation: {:.2f}'.format(sdev))

portfolio return: 0.00009
portfolio standard deviation: 0.22


In [14]:

def sim2weights_df(weights:list, stock_names:list)->pd.DataFrame:
    """Make dataframe given the portfolios´s weights from the simulations.  
    """
    # initialize weights dataframe with the weights from first simulation
    df_weights = pd.DataFrame(weights[0]).T
    for weights in weights[1:]:
        df_weights = df_weights.append(pd.DataFrame(weights).T, ignore_index=True) 
    # rename columns
    df_weights.columns = stock_names
    return df_weights

def sim2kpi_df(sim_returns:list, sim_standard_deviations:list, sim_sharp_ratios:list)->pd.DataFrame:
    """Generate key performance indicator dataframe from simulation returns, simulation standard deviations and simulation sharp ratios."""
    return pd.DataFrame(data={'portfolio return': sim_returns, 
                               'portfolio standard dev': sim_standard_deviations, 
                               'portfolio sharp ratio': sim_sharp_ratios})

def sim2_df(sim_returns:list, sim_standard_deviations:list, sim_sharp_ratios:list, weights:list, stock_names:list)->pd.DataFrame:
    """Save monte-carlo simulation results in dataframe"""
    sim_weights_df = sim2weights_df(weights, stock_names)
    sim_kpi_df = sim2kpi_df(sim_returns, sim_standard_deviations, sim_sharp_ratios) 
    return sim_weights_df.join(sim_kpi_df)

In [15]:
# Monte-Carlo Simulation
w = []
returns = []
stds = []
srs = []

# caclulate normalized daily returns 
df_returns = normalized_returns(df)

for n in range(10000):
    weights = weight_creator(df_returns)
    portfolio_return = portfolio_returns(df_returns, weights)
    portfolio_stdev = portfolio_std(df_returns, weights) 
    portfolio_sr = portfolio_sharp_ratio(portfolio_return, portfolio_stdev, rfr=0.02)
    w.append(weights)
    returns.append(portfolio_return)
    stds.append(portfolio_stdev)
    srs.append(portfolio_sr)

In [16]:
# save simulation results in dataframe
stock_names = list(df.columns)
df_simulation = sim2_df(returns, stds, srs, w, stock_names)
df_simulation.head()

,DAX,Linde,SAP,Deutsche Telekom,Volkswagen,Siemens,Merck,Airbus,Mercedes Benz,Bayer,...,Siemens Energy,HeidelbergCement,Puma,MTU Aero Engines,Covestro,Zalando,HelloFresh,portfolio return,portfolio standard dev,portfolio sharp ratio
0,0.000240,0.049774,0.025645,0.040614,0.009642,0.039824,0.037162,0.041795,0.043317,0.001904,...,0.028534,0.006721,0.038905,0.004972,0.050887,0.015260,0.007294,0.000182,0.227444,0.000447
1,0.051504,0.006685,0.017806,0.047896,0.046015,0.048199,0.051702,0.035605,0.032569,0.007825,...,0.010752,0.037258,0.019323,0.018440,0.032083,0.042693,0.036902,0.000145,0.210395,0.000309
2,0.038846,0.031885,0.050777,0.030135,0.029535,0.007899,0.012797,0.030558,0.007203,0.037065,...,0.051755,0.005765,0.018656,0.029785,0.049537,0.004073,0.010847,0.000061,0.217259,-0.000087
3,0.000500,0.016786,0.032619,0.005861,0.001595,0.001136,0.024142,0.007684,0.045380,0.044543,...,0.009650,0.037918,0.041943,0.021393,0.026264,0.021447,0.011454,0.000175,0.220774,0.000432
4,0.026952,0.021464,0.041604,0.041107,0.040374,0.009123,0.032461,0.000802,0.004887,0.042240,...,0.043361,0.026182,0.032773,0.002508,0.021659,0.023145,0.006633,0.000164,0.213146,0.000392


In [17]:
# plot returns vs risk
fig = px.scatter(x=df_simulation['portfolio standard dev'], 
                 y=df_simulation['portfolio return']*100,
                 color=df_simulation['portfolio sharp ratio']*100,
                 labels={'y': 'return [%]', 'x': 'standard deviation', 'color': 'sharp ratio'},
                 width=600, 
                 title='Portfolio´s Returns and Risks Monte-Carlo Simulation')
fig.show()

In [18]:
# find mimimum risk (i.e. variance) portfolio
def min_variance_portfolio(df_simulation:pd.DataFrame, stocknames:list)->pd.DataFrame:
    """Get weights of the portfolio with minimal variance or risk. Input dataframe of the simulation and a list of the stock names"""
    return df_simulation.sort_values(by='portfolio standard dev')[stocknames].iloc[0,:].values

In [19]:
weights_opt = min_variance_portfolio(df_simulation, stock_names)

In [20]:
# testing the functions
df_returns = normalized_returns(df)
sdev = portfolio_std(df_returns, weights_opt)
returns = portfolio_returns(df_returns, weights_opt)

print('portfolio return: {:.5f}'.format(returns*100))
print('portfolio standard deviation: {:.2f}'.format(sdev))

portfolio return: 0.01457
portfolio standard deviation: 0.19


In [21]:
# find optimal portfolio for given risk

# define risk bounds i.e. maximal acceptable risk
risk_bounds = (0, 0.2)
risk_indices = df_simulation['portfolio standard dev'].between(risk_bounds[0], risk_bounds[1])

# find optimal weights
weights_opt = df_simulation[risk_indices].sort_values(by='portfolio return', ascending=False)[stock_names].iloc[0,:].values
weights_opt

array([0.03193071, 0.011037  , 0.02115925, 0.00442063, 0.0190634 ,
       0.02577434, 0.00596088, 0.02909738, 0.00018301, 0.01310823,
       0.00336484, 0.02747109, 0.04606175, 0.01109653, 0.03654166,
       0.04090042, 0.00331594, 0.05092982, 0.03966646, 0.01898225,
       0.03512196, 0.02243421, 0.04323767, 0.00473128, 0.01950247,
       0.02985381, 0.03410322, 0.03373319, 0.05885551, 0.03238459,
       0.0427613 , 0.04233057, 0.05296261, 0.03397719, 0.02347918,
       0.03154253, 0.00415774, 0.0147654 ])

In [22]:
# testing the functions
df_returns = normalized_returns(df)
sdev = portfolio_std(df_returns, weights_opt)
returns = portfolio_returns(df_returns, weights_opt)

print('portfolio return: {:.5f}'.format(returns*100))
print('portfolio standard deviation: {:.2f}'.format(sdev))

portfolio return: 0.02582
portfolio standard deviation: 0.20
